In [1]:
import plotly.express as px

In [2]:
pip install Faker

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:

import numpy as np


In [4]:
pip install plotly

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [6]:

import pandas as pd
import numpy as np
from faker import Faker



In [37]:
#Create 

# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 100

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hours variable
opening_hours_categories = ['normal working hours', 'always open', 'sport hours']
opening_hours = np.random.choice(opening_hours_categories, size=num_rows)

# Add the opening hours variable to the DataFrame
practice_df['Opening Hours'] = opening_hours

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)
practice_df['aed_coordinates'] = list(zip(practice_df['Latitude'], practice_df['Longitude']))

#generate aed placement variable

aed_placement_categories = ['outside', 'inside']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Display the first few rows of the DataFrame
print(practice_df.head())




          Opening Hours   Latitude  Longitude  \
0  normal working hours  50.511549   4.016955   
1           always open  49.966858   3.538471   
2  normal working hours  50.441391   3.512960   
3           always open  49.923985   4.390353   
4           always open  50.430053   5.623719   

                            aed_coordinates aed_placement  
0    (50.51154925206388, 4.016955350339335)       outside  
1    (49.96685763244193, 3.538470782753567)        inside  
2  (50.441391057425115, 3.5129597960493943)       outside  
3   (49.923985198490506, 4.390352932153449)       outside  
4   (50.430053105498615, 5.623718872934989)       outside  


In [34]:
print(practice_df.columns)

Index(['Opening Hours', 'Latitude', 'Longitude', 'aed_coordinates',
       'aed_placement'],
      dtype='object')


In [45]:
# Assuming 'practice_df' and other variables are defined previously
import plotly.express as px

px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 6       # Zoom level (1-20)

# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
aed_location_layer = px.scatter_mapbox(practice_df, lat='Latitude', lon='Longitude')

# Add the scatter plot layer to the base map
map_with_aed_points = map_base.add_trace(aed_location_layer.data[0])

# Define steps for the slider
steps = []

for timeslider in range(0, 24):    
    colors = []  # Store colors for each point

    for index, row in practice_df.iterrows():
        hours = row['Opening Hours']
        if hours == 'normal working hours' and 0 <= timeslider <= 17: 
            color = 'green'
        elif hours == 'sport hours' and 9 <= timeslider <= 21:
            color = 'green'
        elif hours == 'always open':
            color = 'green'
        else:
            color = 'red'

        colors.append(color)

    # Create step for the current hour
    step = dict(
        method='update',
        args=[{'marker.color': [colors]}],
        label=f'{timeslider}:00'
    )
    steps.append(step)

# Define slider
sliders = [dict(
    active=0,
    pad={"t": 50},
    steps=steps
)]

# Update layout with slider
map_with_aed_points.update_layout(sliders=sliders)

# Show the map
map_with_aed_points.show()